In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
plt.rcParams['figure.figsize'] = (20, 20)

import os
import itertools
import numpy as np
import pandas as pd
from PIL import Image
from scipy.spatial.distance import cosine

from tqdm import tqdm_notebook as tqdm

In [ ]:
size = 20
n_images = size ** 2
path_to_images = '../data/small_images/'

image_ids = np.random.choice(os.listdir(path_to_images), n_images)
images = [Image.open(path_to_images + image_id) for image_id in tqdm(image_ids)]
images = [Image.fromarray(np.stack((image,)*3, -1))
          if len(np.array(image).shape) != 3 else image
          for image in images]

In [ ]:
image_dict = dict(zip(image_ids, images))

similarity = pd.DataFrame(data=[[cosine(image_1.histogram(), image_2.histogram())
                                 for image_1 in images] for image_2 in tqdm(images)],
                          index=image_ids,
                          columns=image_ids)

In [ ]:
sns.heatmap(similarity);

In [ ]:
resolution = 300
    
height = int(resolution * size)
width = int(resolution * size)

big_image = np.empty((height, width, 3)).astype(np.uint8)
grid = np.array(list(itertools.product(range(size), range(size))))
sq_images = [image.resize((300, 300)) for image in images]

for pos, image in zip(grid, sq_images):
    block_t, block_l = pos * resolution
    block_b, block_r = (pos + 1) * resolution
    
    big_image[block_t : block_b, block_l : block_r] = np.array(image)

Image.fromarray(big_image)

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

In [ ]:
most_similar_ids = similarity[query_id].sort_values().index.values[1:6]
similar_images = [image_dict[id].resize((resolution, resolution)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(resolution, 5*resolution, 3))